In [1]:
import csv
import pandas as pd
import numpy as np
from pandas import Series
import os
import timeit
import ipyparallel

In [2]:
aStock = pd.read_csv('apple.2011.csv')
aStock.columns = ['Date','Price','Daily_change'] # Daily_change in Percentage
# Print Content
aStock.head()

,Date,Price,Daily_change
0,1/3/2011,329.57,XXXXX
1,1/4/2011,331.29,0.005218922
2,1/5/2011,334.00,0.008180144
3,1/6/2011,333.73,-0.000808383
4,1/7/2011,336.12,0.007161478


In [3]:
# Print dimension
print aStock.shape

(252, 3)


In [4]:
p = Series(aStock['Daily_change'][1:],dtype=float) # Ignore Header [1:]
mu, sigma = np.mean(p), np.std(p)
mu, sigma

(0.00095735520717131495, 0.016487613979893453)

In [5]:
# Monte Carlo Method

f_price = []
for i in range(0,10000): # Run simulation 10000 times
    l_price = aStock['Price'][251] # Ignore header, hence 251
    diff_20 = np.random.normal(mu, sigma, 20)
    next_20 = [] 
    for e in diff_20:
        l_price = l_price + l_price * e # Notation as shared by David Stern
        next_20.append(l_price)
    f_price.append(next_20[19])

print np.percentile(f_price,1)

346.076110394


Timing the above Monte Carlo method using timeit

In [6]:
%timeit diff_20
f_price = []
for i in range(0,10000):
    l_price = aStock['Price'][251] 
    diff_20 = np.random.normal(mu, sigma, 20)
    next_20 = [] 
    for e in diff_20:
        l_price = l_price + l_price * e # Notation as shared by David Stern
        next_20.append(l_price)
    f_price.append(next_20[19])

print np.percentile(f_price,1)

10000000 loops, best of 3: 21.3 ns per loop
345.87685364


In [7]:
from ipyparallel import Client
c = Client()
clients = ipyparallel.Client()
dview = clients.direct_view()

In [8]:
%%px 
import csv
import pandas as pd
import numpy as np
from pandas import Series
import os
import timeit
import ipyparallel

aStock = pd.read_csv('apple.2011.csv')
aStock.columns = ['Date','Price','Daily_change'] # Daily_change in Percentage

# Analysis from HW11
p = Series(aStock['Daily_change'][1:],dtype=float) # Ignore Header [1:]
mu, sigma = np.mean(p), np.std(p)
mu, sigma

# the parallel part
clients = ipyparallel.Client()
dview = clients.direct_view()

def rand20(mean,std):
    import random
    randomlist = []
    for i in range(0,20):
         randomlist.append(random.gauss(mean,std))
    return randomlist

In [9]:
# How many cluster ? 4
c.ids

[0, 1, 2, 3]

In [10]:
%%px 
f_price = []
diff_20 = []

%timeit diff_20
for i in range(0,10000):
    l_price = aStock['Price'][251] 
    diff_20 = np.random.normal(mu, sigma, 20)
    next_20 = [] 
    for e in diff_20:
        l_price = l_price + l_price * e 
        next_20.append(l_price)
    f_price.append(next_20[19])

print np.percentile(f_price,1)

[stdout:0] 
The slowest run took 48.24 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 21.5 ns per loop
347.611259525
[stdout:1] 
10000000 loops, best of 3: 22 ns per loop
346.688363322
[stdout:2] 
10000000 loops, best of 3: 22.1 ns per loop
348.309032024
[stdout:3] 
The slowest run took 87.90 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 21.7 ns per loop
346.983123799
